# Notebook 01: Pipeline d'Ingestion et d'Embedding des Données ArXiv

Ce notebook démontre le processus complet d'acquisition, de traitement, d'embedding et de stockage des articles scientifiques d'ArXiv dans notre base de données MongoDB.

**Prérequis :**
* Assurez-vous d'avoir exécuté le notebook `00_setup_environment.ipynb` et que votre environnement est correctement configuré (variables d'environnement chargées, clés API valides, MongoDB accessible).
* Les bibliothèques nécessaires doivent être installées via `environment.yml`.

**Étapes de ce Notebook :**
1.  Configuration initiale (imports, logging, connexion MongoDB).
2.  Téléchargement d'articles depuis ArXiv.
3.  Parsing des documents PDF et de leurs métadonnées.
4.  Prétraitement du texte (nettoyage et chunking).
5.  Génération des embeddings pour les chunks.
6.  Stockage des chunks et de leurs embeddings dans MongoDB et création des index.

In [3]:
# Importer nos modules
import logging
from typing import List, Optional, Dict, Any
from pathlib import Path # Assurez-vous que Path est importé
import re # Pour nettoyer le nom du répertoire du corpus

from pymongo.errors import ConnectionFailure 
from config.settings import settings
from config.logging_config import setup_logging
# Les fonctions attendent maintenant les chemins complets en argument
from src.data_processing.arxiv_downloader import download_pipeline as download_arxiv_papers
from src.data_processing.document_parser import parse_document_collection 
# PDF_INPUT_DIR et METADATA_INPUT_DIR ont été supprimés de document_parser.py

from src.data_processing.preprocessor import preprocess_parsed_documents, ParsedDocument 
from src.data_processing.embedder import generate_embeddings_for_chunks 
from src.vector_store.mongodb_manager import MongoDBManager

# Configurer le logging pour le notebook
setup_logging(level="INFO") # Mettre à DEBUG pour plus de détails
logger = logging.getLogger("nb_01_ingestion_embedding")

logger.info("Configuration initiale du notebook terminée.")

# --- Définition dynamique des chemins pour ce notebook ---
# Utiliser la requête du notebook pour créer un nom de répertoire de corpus unique
ARXIV_QUERY_NOTEBOOK = "explainable artificial intelligence for robotics" # Gardé pour la requête ArXiv
# Fonction simple pour nettoyer la chaîne de requête pour un nom de répertoire
def sanitize_for_dir_name(query: str) -> str:
    s = query.lower()
    s = re.sub(r'[\s\W-]+', '_', s)
    s = s.strip('_')
    return s[:50]

# Nom du sous-répertoire de corpus basé sur la requête spécifique à ce notebook
corpus_sub_dir_name_nb = sanitize_for_dir_name(ARXIV_QUERY_NOTEBOOK) 
# Ou utilisez un nom fixe si vous préférez pour les tests du notebook, par ex :
# corpus_sub_dir_name_nb = "notebook_ingestion_test_corpus"

notebook_corpus_base_path = Path(settings.DATA_DIR) / "corpus" / corpus_sub_dir_name_nb
# Définir les chemins spécifiques pour les PDFs et métadonnées de ce notebook
# Ces variables remplaceront les anciennes PDF_INPUT_DIR et METADATA_INPUT_DIR dans ce notebook
NB_PDF_OUTPUT_DIR = notebook_corpus_base_path / "pdfs"
NB_METADATA_OUTPUT_DIR = notebook_corpus_base_path / "metadata"

# S'assurer que ces répertoires existent pour le notebook
NB_PDF_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
NB_METADATA_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

logger.info(f"Utilisation de DATA_DIR: {settings.DATA_DIR}")
logger.info(f"Pour ce notebook, les PDFs ArXiv seront gérés dans : {NB_PDF_OUTPUT_DIR}")
logger.info(f"Pour ce notebook, les métadonnées ArXiv seront gérées dans : {NB_METADATA_OUTPUT_DIR}")

# --- Affichage de la configuration d'embedding active et vérification des prérequis (inchangé) ---
logger.info(f"--- Configuration d'Embedding Active (depuis settings.py et .env) ---")
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Fournisseur d'embedding par défaut configuré : {active_embedding_provider}")

if active_embedding_provider == "openai":
    logger.info(f"  Modèle OpenAI Embedding à utiliser : {settings.OPENAI_EMBEDDING_MODEL_NAME}")
    logger.info(f"  Dimension OpenAI Embedding (configurée) : {settings.OPENAI_EMBEDDING_DIMENSION}")
    if not settings.OPENAI_API_KEY:
        logger.error("ERREUR : Le fournisseur d'embedding est 'openai', mais OPENAI_API_KEY n'est pas configurée dans .env. La génération d'embeddings échouera.")
elif active_embedding_provider == "huggingface":
    logger.info(f"  Modèle HuggingFace Embedding à utiliser : {settings.HUGGINGFACE_EMBEDDING_MODEL_NAME}")
    logger.info(f"  Dimension HuggingFace Embedding (configurée) : {settings.HUGGINGFACE_EMBEDDING_MODEL_DIMENSION}")
elif active_embedding_provider == "ollama":
    logger.info(f"  Modèle Ollama Embedding à utiliser : {settings.OLLAMA_EMBEDDING_MODEL_NAME}")
    logger.info(f"  Dimension Ollama Embedding (configurée) : {settings.OLLAMA_EMBEDDING_MODEL_DIMENSION}")
    logger.info(f"  URL de base Ollama : {settings.OLLAMA_BASE_URL}")
    if not settings.OLLAMA_BASE_URL:
        logger.error("ERREUR : Le fournisseur d'embedding est 'ollama', mais OLLAMA_BASE_URL n'est pas configurée. La génération d'embeddings échouera.")
    logger.info(f"  ASSUREZ-VOUS que le modèle '{settings.OLLAMA_EMBEDDING_MODEL_NAME}' est disponible sur votre serveur Ollama (ex: via 'ollama pull {settings.OLLAMA_EMBEDDING_MODEL_NAME}').")
else:
    logger.error(f"ERREUR : Fournisseur d'embedding inconnu configuré dans settings.py : '{active_embedding_provider}'. La génération d'embeddings échouera.")
# --- Fin de la section sur la configuration d'embedding ---

# Paramètres pour cette exécution de notebook (inchangés, mais les chemins de sortie seront NB_PDF_OUTPUT_DIR etc.)
# ARXIV_QUERY_NOTEBOOK est défini plus haut
MAX_RESULTS_NOTEBOOK = 2 
COLLECTION_NAME_NOTEBOOK = "arxiv_chunks_notebook_test" 
VECTOR_INDEX_NAME_NOTEBOOK = "vector_index_notebook_test"
TEXT_INDEX_NAME_NOTEBOOK = "text_index_notebook_test"

# La section de nettoyage des répertoires (si décommentée) devra aussi utiliser NB_PDF_OUTPUT_DIR et NB_METADATA_OUTPUT_DIR
# import shutil
# if NB_PDF_OUTPUT_DIR.exists():
#     logger.info(f"Nettoyage du répertoire PDF: {NB_PDF_OUTPUT_DIR}")
#     shutil.rmtree(NB_PDF_OUTPUT_DIR)
# NB_PDF_OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # Recréer après suppression
# if NB_METADATA_OUTPUT_DIR.exists():
#     logger.info(f"Nettoyage du répertoire Metadata: {NB_METADATA_OUTPUT_DIR}")
#     shutil.rmtree(NB_METADATA_OUTPUT_DIR)
# NB_METADATA_OUTPUT_DIR.mkdir(parents=True, exist_ok=True) # Recréer après suppression

2025-06-02 22:48:12 - nb_01_ingestion_embedding - INFO - Configuration initiale du notebook terminée.
2025-06-02 22:48:12 - nb_01_ingestion_embedding - INFO - Utilisation de DATA_DIR: /home/facetoface/cognitive-swarm-agents/data
2025-06-02 22:48:12 - nb_01_ingestion_embedding - INFO - Pour ce notebook, les PDFs ArXiv seront gérés dans : /home/facetoface/cognitive-swarm-agents/data/corpus/explainable_artificial_intelligence_for_robotics/pdfs
2025-06-02 22:48:12 - nb_01_ingestion_embedding - INFO - Pour ce notebook, les métadonnées ArXiv seront gérées dans : /home/facetoface/cognitive-swarm-agents/data/corpus/explainable_artificial_intelligence_for_robotics/metadata
2025-06-02 22:48:12 - nb_01_ingestion_embedding - INFO - --- Configuration d'Embedding Active (depuis settings.py et .env) ---
2025-06-02 22:48:12 - nb_01_ingestion_embedding - INFO - Fournisseur d'embedding par défaut configuré : ollama
2025-06-02 22:48:12 - nb_01_ingestion_embedding - INFO -   Modèle Ollama Embedding à util

### Étape 1: Téléchargement d'Articles depuis ArXiv

Nous utilisons `arxiv_downloader.download_pipeline` pour rechercher et télécharger quelques articles.
Pour ce notebook, nous limitons la recherche à `MAX_RESULTS_NOTEBOOK` articles pour que l'exécution soit rapide.

In [5]:
logger.info(f"--- Étape 1: Téléchargement d'Articles ArXiv (max: {MAX_RESULTS_NOTEBOOK}) ---")

# S'assurer que NB_PDF_OUTPUT_DIR et NB_METADATA_OUTPUT_DIR sont bien définis
# depuis la première cellule de code de ce notebook.
# Si ce n'est pas le cas, vous pouvez les redéfinir ici ou vous assurer que la première cellule a été exécutée.
# Exemple (au cas où, à adapter si vous avez utilisé un autre nom pour le corpus du notebook) :
# from pathlib import Path
# from config.settings import settings
# import re
# def sanitize_for_dir_name(query: str) -> str:
#     s = query.lower()
#     s = re.sub(r'[\s\W-]+', '_', s)
#     s = s.strip('_')
#     return s[:50]
# corpus_sub_dir_name_nb = sanitize_for_dir_name(ARXIV_QUERY_NOTEBOOK) 
# notebook_corpus_base_path = Path(settings.DATA_DIR) / "corpus" / corpus_sub_dir_name_nb
# NB_PDF_OUTPUT_DIR = notebook_corpus_base_path / "pdfs"
# NB_METADATA_OUTPUT_DIR = notebook_corpus_base_path / "metadata"
# NB_PDF_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
# NB_METADATA_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


downloaded_files_info = download_arxiv_papers(
    query=ARXIV_QUERY_NOTEBOOK,
    max_results=MAX_RESULTS_NOTEBOOK,
    # MODIFICATION : Ajout des arguments requis pour les chemins de sortie
    pdf_output_dir=NB_PDF_OUTPUT_DIR,
    metadata_output_dir=NB_METADATA_OUTPUT_DIR
    # Les arguments sort_by et sort_order utiliseront leurs valeurs par défaut
    # définies dans la fonction download_pipeline si non spécifiés ici.
)

if downloaded_files_info and downloaded_files_info.get('pdfs'):
    logger.info(f"Téléchargement terminé. {len(downloaded_files_info['pdfs'])} PDFs et {len(downloaded_files_info['metadata'])} fichiers de métadonnées.")
    for pdf_path in downloaded_files_info['pdfs'][:2]: # Afficher les 2 premiers
        logger.info(f"  PDF téléchargé : {pdf_path}")
    for meta_path in downloaded_files_info['metadata'][:2]:
        logger.info(f"  Métadonnées sauvegardées : {meta_path}")
else:
    logger.warning("Aucun fichier PDF n'a été téléchargé. Vérifiez la requête ArXiv ou la connexion.")
    # On pourrait arrêter ici si aucun fichier n'est téléchargé

2025-06-02 22:49:21 - nb_01_ingestion_embedding - INFO - --- Étape 1: Téléchargement d'Articles ArXiv (max: 2) ---
2025-06-02 22:49:21 - src.data_processing.arxiv_downloader - INFO - Searching ArXiv with query='explainable artificial intelligence for robotics', max_results=2, sort_by='submittedDate', sort_order='descending'
2025-06-02 22:49:21 - arxiv - INFO - Requesting page (first: True, try: 0): https://export.arxiv.org/api/query?search_query=explainable+artificial+intelligence+for+robotics&id_list=&sortBy=submittedDate&sortOrder=descending&start=0&max_results=100
2025-06-02 22:49:24 - arxiv - INFO - Got first page: 100 of 2237800 total results
2025-06-02 22:49:24 - src.data_processing.arxiv_downloader - INFO - Found 2 papers on ArXiv.
2025-06-02 22:49:24 - src.data_processing.arxiv_downloader - INFO - Starting download and metadata saving for 2 papers.
2025-06-02 22:49:24 - src.data_processing.arxiv_downloader - INFO - Processing paper 1/2: http://arxiv.org/abs/2505.24878v1
2025-06

### Étape 2: Parsing des Documents

Maintenant, nous utilisons `document_parser.parse_document_collection` pour lire les PDFs téléchargés et leurs fichiers de métadonnées JSON associés. Cela extraira le texte brut et structurera les métadonnées.

In [7]:
logger.info(f"\n--- Étape 2: Parsing des Documents ---")

# S'assurer que NB_PDF_OUTPUT_DIR et NB_METADATA_OUTPUT_DIR sont bien définis
# depuis la première cellule de code de ce notebook.
# Ces variables pointent vers les répertoires où download_arxiv_papers
# a (ou aurait dû) sauvegarder les fichiers à l'étape précédente.

# MODIFICATION : Passer les chemins corrects à parse_document_collection
parsed_documents: List[ParsedDocument] = parse_document_collection(
    pdf_dir=NB_PDF_OUTPUT_DIR,
    metadata_dir=NB_METADATA_OUTPUT_DIR
)

if parsed_documents:
    logger.info(f"{len(parsed_documents)} documents ont été parsés avec succès.")
    # Afficher un extrait du premier document parsé
    if len(parsed_documents) > 0:
        doc_example = parsed_documents[0]
        logger.info(f"Exemple de document parsé (ID ArXiv: {doc_example['arxiv_id']}):")
        logger.info(f"  Titre (depuis métadonnées): {doc_example['metadata'].get('title', 'N/A')}")
        logger.info(f"  Extrait du texte: '{doc_example['text_content'][:200].replace(chr(10), ' ')}...'")
        logger.info(f"  Chemin PDF: {doc_example['pdf_path']}")
        logger.info(f"  Chemin Métadonnées: {doc_example['metadata_path']}")
else:
    logger.warning("Aucun document n'a été parsé. Vérifiez si des PDFs existent dans le répertoire attendu et si l'étape précédente de téléchargement a réussi.")

2025-06-02 22:50:55 - nb_01_ingestion_embedding - INFO - 
--- Étape 2: Parsing des Documents ---
2025-06-02 22:50:55 - src.data_processing.document_parser - INFO - Found 2 PDF files in /home/facetoface/cognitive-swarm-agents/data/corpus/explainable_artificial_intelligence_for_robotics/pdfs to parse.
2025-06-02 22:50:55 - src.data_processing.document_parser - INFO - Parsing PDF: 2505.24878.pdf
2025-06-02 22:50:55 - src.data_processing.document_parser - INFO - Successfully processed and added document: 2505.24878
2025-06-02 22:50:55 - src.data_processing.document_parser - INFO - Parsing PDF: 2505.24877.pdf
2025-06-02 22:50:56 - src.data_processing.document_parser - INFO - Successfully processed and added document: 2505.24877
2025-06-02 22:50:56 - src.data_processing.document_parser - INFO - Finished parsing collection. Successfully processed 2 documents.
2025-06-02 22:50:56 - nb_01_ingestion_embedding - INFO - 2 documents ont été parsés avec succès.
2025-06-02 22:50:56 - nb_01_ingestion_

### Étape 3: Prétraitement du Texte (Nettoyage et Chunking)

Les documents parsés sont maintenant nettoyés et découpés en chunks plus petits et gérables en utilisant `preprocessor.preprocess_parsed_documents`.

In [8]:
logger.info(f"\n--- Étape 3: Prétraitement du Texte ---")

if parsed_documents:
    processed_chunks = preprocess_parsed_documents(parsed_documents)
    if processed_chunks:
        logger.info(f"{len(processed_chunks)} chunks ont été générés après prétraitement.")
        # Afficher un extrait du premier chunk
        if len(processed_chunks) > 0:
            chunk_example = processed_chunks[0]
            logger.info(f"Exemple de chunk traité (ID: {chunk_example['chunk_id']}):")
            logger.info(f"  ID ArXiv d'origine: {chunk_example['arxiv_id']}")
            logger.info(f"  Titre d'origine: {chunk_example['original_document_title']}")
            logger.info(f"  Extrait du chunk: '{chunk_example['text_chunk'][:200].replace(chr(10), ' ')}...'")
    else:
        logger.warning("Aucun chunk n'a été généré lors du prétraitement.")
else:
    logger.warning("Aucun document parsé à prétraiter. Étape de prétraitement sautée.")
    processed_chunks = [] # S'assurer que la variable existe

2025-06-02 22:51:04 - nb_01_ingestion_embedding - INFO - 
--- Étape 3: Prétraitement du Texte ---
2025-06-02 22:51:04 - src.data_processing.preprocessor - INFO - Starting preprocessing for 2 parsed documents.
2025-06-02 22:51:04 - src.data_processing.preprocessor - INFO - Preprocessing document 1/2: 2505.24878
2025-06-02 22:51:04 - src.data_processing.preprocessor - INFO - Preprocessing document 2/2: 2505.24877
2025-06-02 22:51:04 - src.data_processing.preprocessor - INFO - Finished preprocessing. Generated 43 chunks in total.
2025-06-02 22:51:04 - nb_01_ingestion_embedding - INFO - 43 chunks ont été générés après prétraitement.
2025-06-02 22:51:04 - nb_01_ingestion_embedding - INFO - Exemple de chunk traité (ID: 2505.24878_chunk_001):
2025-06-02 22:51:04 - nb_01_ingestion_embedding - INFO -   ID ArXiv d'origine: 2505.24878
2025-06-02 22:51:04 - nb_01_ingestion_embedding - INFO -   Titre d'origine: Open CaptchaWorld: A Comprehensive Web-based Platform for Testing and Benchmarking Multi

### Étape 4: Génération des Embeddings

Chaque chunk de texte est maintenant converti en une représentation vectorielle (embedding) en utilisant `embedder.generate_embeddings_for_chunks`.

In [9]:
# Contenu de la cellule avec l'ID 817ea79a (Étape 4)
logger.info(f"\n--- Étape 4: Génération des Embeddings ---")
# MODIFIÉ : L'annotation de type pour chunks_with_embeddings
chunks_with_embeddings: List[Dict[str, Any]] = [] # Anciennement List[ProcessedChunkWithEmbedding]

if processed_chunks:
    # Vérification proactive des prérequis pour le fournisseur d'embedding configuré
    provider_check_ok = True
    active_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
    if active_provider == "openai" and not settings.OPENAI_API_KEY:
        logger.error("OpenAI est le fournisseur d'embedding, mais OPENAI_API_KEY n'est pas configurée. Impossible de générer les embeddings.")
        provider_check_ok = False
    elif active_provider == "ollama":
        if not settings.OLLAMA_BASE_URL:
            logger.error("Ollama est le fournisseur d'embedding, mais OLLAMA_BASE_URL n'est pas configurée.")
            provider_check_ok = False
        if not settings.OLLAMA_EMBEDDING_MODEL_NAME:
            logger.error("Ollama est le fournisseur d'embedding, mais OLLAMA_EMBEDDING_MODEL_NAME n'est pas configuré.")
            provider_check_ok = False
    
    if provider_check_ok:
        logger.info(f"Appel de generate_embeddings_for_chunks avec le provider: {active_provider}")
        # generate_embeddings_for_chunks retourne maintenant List[Dict[str, Any]]
        chunks_with_embeddings = generate_embeddings_for_chunks(processed_chunks)
        
        if chunks_with_embeddings:
            logger.info(f"{len(chunks_with_embeddings)} chunks ont maintenant des embeddings.")
            if len(chunks_with_embeddings) > 0:
                chunk_emb_example = chunks_with_embeddings[0] # C'est maintenant un Dict[str, Any]
                logger.info(f"Exemple de chunk avec embedding (ID: {chunk_emb_example['chunk_id']}):")
                # MODIFIÉ : Accès aux champs via le sous-dictionnaire 'metadata'
                logger.info(f"  Fournisseur d'Embedding Utilisé: {chunk_emb_example['metadata'].get('embedding_provider', 'N/A')}")
                logger.info(f"  Modèle d'Embedding Utilisé: {chunk_emb_example['metadata'].get('embedding_model', 'N/A')}")
                logger.info(f"  Dimension Réelle de l'Embedding: {chunk_emb_example['metadata'].get('embedding_dimension', 'N/A')}") 
                logger.info(f"  Vecteur d'Embedding (5 premières dimensions): {chunk_emb_example.get('embedding', [])[:5]}...") # 'embedding' est toujours au premier niveau
        else:
            logger.warning("Aucun embedding n'a été généré.")
    else:
        logger.warning("Prérequis non remplis pour le fournisseur d'embedding configuré. Étape d'embedding sautée.")
else:
    logger.warning("Aucun chunk traité à embedder. Étape d'embedding sautée.")

2025-06-02 22:51:08 - nb_01_ingestion_embedding - INFO - 
--- Étape 4: Génération des Embeddings ---
2025-06-02 22:51:08 - nb_01_ingestion_embedding - INFO - Appel de generate_embeddings_for_chunks avec le provider: ollama
2025-06-02 22:51:08 - src.data_processing.embedder - INFO - Starting embedding generation for 43 chunks.
2025-06-02 22:51:08 - src.data_processing.embedder - INFO - Initializing embedding client for provider: ollama
2025-06-02 22:51:08 - src.data_processing.embedder - INFO - Using OllamaEmbeddings with model: nomic-embed-text via http://localhost:11434
2025-06-02 22:51:08 - src.data_processing.embedder - INFO - Embedding batch 1/2 (size: 32) using ollama provider.


/home/facetoface/cognitive-swarm-agents/src/data_processing/embedder.py:51: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  return OllamaEmbeddings(


2025-06-02 22:51:09 - src.data_processing.embedder - INFO - Embedding batch 2/2 (size: 11) using ollama provider.
2025-06-02 22:51:10 - src.data_processing.embedder - INFO - Finished embedding generation. Successfully structured 43 chunks for DB out of 43.
2025-06-02 22:51:10 - nb_01_ingestion_embedding - INFO - 43 chunks ont maintenant des embeddings.
2025-06-02 22:51:10 - nb_01_ingestion_embedding - INFO - Exemple de chunk avec embedding (ID: 2505.24878_chunk_001):
2025-06-02 22:51:10 - nb_01_ingestion_embedding - INFO -   Fournisseur d'Embedding Utilisé: ollama
2025-06-02 22:51:10 - nb_01_ingestion_embedding - INFO -   Modèle d'Embedding Utilisé: nomic-embed-text
2025-06-02 22:51:10 - nb_01_ingestion_embedding - INFO -   Dimension Réelle de l'Embedding: 768
2025-06-02 22:51:10 - nb_01_ingestion_embedding - INFO -   Vecteur d'Embedding (5 premières dimensions): [0.1418522745370865, 1.1173579692840576, -2.4316766262054443, -0.11591464281082153, 1.5562331676483154]...


### Étape 5: Stockage dans MongoDB et Création des Index

Enfin, les chunks avec leurs embeddings sont insérés dans une collection MongoDB. Nous créons également les index de recherche vectorielle et textuelle nécessaires pour notre moteur RAG.

In [10]:
# Cellule Étape 5: Stockage MongoDB et Création d'Index (ID: ecc36835)

logger.info(f"\n--- Étape 5: Stockage MongoDB et Création d'Index ---")
mongo_mgr = None 

# chunks_with_embeddings est une List[Dict[str, Any]] avec la nouvelle structure
if chunks_with_embeddings:
    try:
        logger.info(f"Initialisation de MongoDBManager pour la collection: {COLLECTION_NAME_NOTEBOOK}")
        mongo_mgr = MongoDBManager(mongo_uri=settings.MONGODB_URI, db_name=settings.MONGO_DATABASE_NAME)
        mongo_mgr.connect()

        test_collection = mongo_mgr.get_collection(COLLECTION_NAME_NOTEBOOK)
        if test_collection is not None: 
            logger.info(f"Suppression des documents existants dans la collection de test '{COLLECTION_NAME_NOTEBOOK}'...")
            delete_result = test_collection.delete_many({})
            logger.info(f"{delete_result.deleted_count} documents supprimés.")
        else:
            # Si test_collection est None, cela signifie que mongo_mgr.get_collection a échoué (probablement un problème de connexion)
            # Il est préférable de ne pas continuer si la collection n'est pas accessible.
            logger.error(f"Impossible d'obtenir la collection {COLLECTION_NAME_NOTEBOOK}. L'insertion et la création d'index vont échouer. Arrêt.")
            # Vous pourriez vouloir lever une exception ici pour arrêter l'exécution du notebook
            raise ConnectionError(f"Impossible d'obtenir la collection {COLLECTION_NAME_NOTEBOOK} depuis MongoDB.")


        logger.info(f"Insertion de {len(chunks_with_embeddings)} chunks dans MongoDB...")
        # insert_chunks_with_embeddings attend List[Dict[str, Any]] et mappe chunk_id à _id
        insertion_summary = mongo_mgr.insert_chunks_with_embeddings(
            chunks_with_embeddings,
            collection_name=COLLECTION_NAME_NOTEBOOK
        )
        logger.info(f"Résumé de l'insertion MongoDB: {insertion_summary}")

        if insertion_summary.get("inserted_count", 0) > 0:
            logger.info(f"Création/Vérification de l'index vectoriel '{VECTOR_INDEX_NAME_NOTEBOOK}'. La dimension de l'index sera basée sur le fournisseur d'embedding configuré: '{settings.DEFAULT_EMBEDDING_PROVIDER}'.")
            
            # MODIFIÉ : Mettre à jour les chemins pour les champs de filtrage vectoriel
            # Ces champs sont maintenant attendus à l'intérieur du champ "metadata"
            vector_filter_fields = [
                "metadata.arxiv_id", 
                "metadata.original_document_title", 
                "metadata.primary_category", # Assurez-vous que 'primary_category' est bien dans le dict 'metadata' des chunks
                "metadata.embedding_provider", 
                "metadata.embedding_model"     
            ]
            # Note: Si 'metadata.primary_category' n'existe pas dans tous les documents, 
            # l'indexation de ce champ spécifique pourrait ne pas s'appliquer à ces documents, 
            # mais la création de l'index devrait réussir si le type est bien géré (e.g. stringFacet).

            success_vector_idx = mongo_mgr.create_vector_search_index(
                collection_name=COLLECTION_NAME_NOTEBOOK,
                index_name=VECTOR_INDEX_NAME_NOTEBOOK, # Doit être le nom de l'index de type "vectorSearch"
                embedding_field="embedding", # Ce champ est au premier niveau
                filter_fields=vector_filter_fields # Ces champs sont maintenant préfixés par "metadata."
            )
            if success_vector_idx:
                logger.info("Index vectoriel géré avec succès.")
            else:
                logger.warning("Problème lors de la gestion de l'index vectoriel.")

            logger.info(f"Création/Vérification de l'index textuel '{TEXT_INDEX_NAME_NOTEBOOK}'...")
            # MODIFIÉ : Mettre à jour les chemins pour les champs de texte additionnels
            additional_text_fields_for_index = {
                "metadata.original_document_title": "string", 
                # Si vous aviez un champ 'title' distinct dans les métadonnées source :
                # "metadata.title": "string", 
                # Assurez-vous que ces chemins existent réellement dans la structure de vos documents MongoDB
            }
            success_text_idx = mongo_mgr.create_text_search_index(
                collection_name=COLLECTION_NAME_NOTEBOOK,
                index_name=TEXT_INDEX_NAME_NOTEBOOK,
                text_field="text_chunk", # Ce champ est au premier niveau
                additional_text_fields=additional_text_fields_for_index
            )
            if success_text_idx:
                logger.info("Index textuel géré avec succès.")
            else:
                logger.warning("Problème lors de la gestion de l'index textuel.")
            
            if test_collection is not None: # Ré-vérifier, même si on a levé une erreur plus tôt
                first_chunk_id_val = chunks_with_embeddings[0]["chunk_id"]
                sample_doc_from_db = test_collection.find_one({"_id": first_chunk_id_val})
                if sample_doc_from_db:
                    logger.info(f"Exemple de document récupéré de MongoDB (ID: {sample_doc_from_db['_id']}):")
                    logger.info(f"  Texte: {sample_doc_from_db.get('text_chunk', '')[:100]}...")
                    
                    # MODIFIÉ : Accès aux champs via le sous-dictionnaire 'metadata'
                    doc_metadata = sample_doc_from_db.get("metadata")
                    if isinstance(doc_metadata, dict):
                        logger.info(f"  Fournisseur Embedding (depuis metadata): {doc_metadata.get('embedding_provider')}")
                        logger.info(f"  Modèle Embedding (depuis metadata): {doc_metadata.get('embedding_model')}")
                        logger.info(f"  Dimension Embedding (depuis metadata): {doc_metadata.get('embedding_dimension')}")
                        logger.info(f"  ArXiv ID (depuis metadata): {doc_metadata.get('arxiv_id')}")
                        logger.info(f"  Titre Doc (depuis metadata): {doc_metadata.get('original_document_title')}")
                        logger.info(f"  Primary Category (depuis metadata): {doc_metadata.get('primary_category', 'N/A')}") # Exemple
                    else:
                        logger.warning("  Le champ 'metadata' est manquant ou n'est pas un dictionnaire dans le document récupéré.")
                    
                    logger.info(f"  Vecteur Embedding (premières dims): {str(sample_doc_from_db.get('embedding', [])[:3])[:100]}...")
                else:
                    logger.warning(f"Impossible de récupérer le document d'exemple '{first_chunk_id_val}' depuis MongoDB.")
            # La clause 'else' pour 'test_collection is None' est déjà gérée par le raise ConnectionError plus haut.
        else:
            logger.warning("Aucun document n'a été inséré, la création des index pourrait ne pas être pertinente ou échouer sur une collection vide.")

    except ConnectionFailure as cf: # Spécifique pour les erreurs de connexion attrapées plus tôt
        logger.error(f"Erreur de connexion MongoDB non récupérable : {cf}", exc_info=True)
        # L'exécution s'arrête ici si ConnectionError a été levée plus tôt
    except Exception as e:
        logger.error(f"Erreur lors des opérations MongoDB: {e}", exc_info=True)
    finally:
        if mongo_mgr:
            mongo_mgr.close()
            logger.info("Connexion MongoDB fermée.")
else:
    logger.warning("Aucun chunk avec embedding à stocker. Étape MongoDB sautée.")

logger.info("\nPipeline d'Ingestion et d'Embedding terminé pour ce notebook !")

2025-06-02 22:51:14 - nb_01_ingestion_embedding - INFO - 
--- Étape 5: Stockage MongoDB et Création d'Index ---
2025-06-02 22:51:14 - nb_01_ingestion_embedding - INFO - Initialisation de MongoDBManager pour la collection: arxiv_chunks_notebook_test
2025-06-02 22:51:14 - src.vector_store.mongodb_manager - INFO - MongoDBManager initialized for database: cognitive_swarm_db
2025-06-02 22:51:14 - src.vector_store.mongodb_manager - INFO - Successfully connected to MongoDB database: cognitive_swarm_db
2025-06-02 22:51:14 - nb_01_ingestion_embedding - INFO - Suppression des documents existants dans la collection de test 'arxiv_chunks_notebook_test'...
2025-06-02 22:51:15 - nb_01_ingestion_embedding - INFO - 0 documents supprimés.
2025-06-02 22:51:15 - nb_01_ingestion_embedding - INFO - Insertion de 43 chunks dans MongoDB...
2025-06-02 22:51:15 - src.vector_store.mongodb_manager - INFO - Attempting to insert 43 chunks into collection 'arxiv_chunks_notebook_test'.
2025-06-02 22:51:16 - src.vecto

## Conclusion

Ce notebook a illustré l'ensemble du pipeline d'ingestion :
- Téléchargement des données sources (ArXiv).
- Parsing pour extraire le texte et les métadonnées.
- Prétraitement pour nettoyer et diviser le texte en chunks.
- Génération des embeddings pour chaque chunk.
- Stockage des données enrichies dans MongoDB et création des index nécessaires pour la recherche.

Les données sont maintenant prêtes à être utilisées par le `RetrievalEngine` et les agents du "Cognitive Swarm". Vous pouvez explorer la collection MongoDB (`arxiv_chunks_notebook_test` dans la base `cognitive_swarm_db` par défaut) pour voir les documents stockés.